<a href="https://colab.research.google.com/github/Angel-20524/CODSOFT/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-surprise

In [ ]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
ratings_df = pd.read_csv('/content/ratings.csv')
movies_df = pd.read_csv('/content/movies.csv')

df = pd.merge(ratings_df, movies_df[['movieId', 'genres']], on='movieId', how='left')
df

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Horror
100834,610,168252,5.0,1493846352,Action|Sci-Fi


In [ ]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
mlb = MultiLabelBinarizer()
df['userId'] = user_encoder.fit_transform(df['userId'])
df['movieId'] = movie_encoder.fit_transform(df['movieId'])
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres').str.split('|')), columns=mlb.classes_, index=df.index))

In [ ]:
df.drop(columns = "(no genres listed)", inplace = True)

In [ ]:
train_df, test_df = train_test_split(df, test_size = 0.2)
train_df

,userId,movieId,rating,timestamp,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
78851,488,3006,2.5,1333101702,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
64552,413,6272,3.0,1216149371,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
44883,297,4124,2.0,1447599792,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5048,32,124,4.0,939715904,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
78826,488,2796,4.5,1333831394,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55179,364,9404,2.5,1489796824,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
81746,516,3002,4.0,1487953207,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48630,313,238,3.0,834241987,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
80640,508,6136,3.0,1436101624,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
reader = Reader(rating_scale = (0.5, 5))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

In [ ]:
model_svd = SVD()
model_svd.fit(trainset)

predictions_svd = model_svd.test(trainset.build_anti_testset())
accuracy.rmse(predictions_svd)

RMSE: 0.4726


0.4726062372159372

In [ ]:
def get_top_n_recommendations(user_id, n=5):
  user_movies = df[df['userId'] == user_id]['movieId'].unique()
  all_movies = df['movieId'].unique()
  movies_to_predict = list(set(all_movies) - set(user_movies))

  user_movie_pairs = [(user_id, movie_id, 0) for movie_id in movies_to_predict]
  predictions_cf = model_svd.test(user_movie_pairs)

  top_n_recommendations = sorted(predictions_cf, key = lambda x: x.est)[:n]

  for pred in top_n_recommendations:
    predicted_rating = pred.est
    print(predicted_rating)


  top_n_movie_ids = [int(pred.iid) for pred in top_n_recommendations]

  top_n_movies = movie_encoder.inverse_transform(top_n_movie_ids)

  return top_n_movies


In [ ]:
user_id = 221
recommendations = get_top_n_recommendations(user_id)
top_n_movies_titles = movies_df[movies_df['movieId'].isin(recommendations)]['title'].tolist()
print(f"Top 5 Recommendations for User {user_id}:")
for i, title in enumerate(top_n_movies_titles, 1):
  print(f"{i}.{title}")

1.623468030680751
1.7229691732197765
1.8277294909507225
1.8332592736393287
1.8781292271974288
Top 5 Recommendations for User 221:
1.I Know What You Did Last Summer (1997)
2.Spice World (1997)
3.I Still Know What You Did Last Summer (1998)
4.Flintstones in Viva Rock Vegas, The (2000)
5.Battlefield Earth (2000)
